In [6]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("processed_car_data.csv")


In [3]:
data.head()

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company_name
0,3,1,0,2,0,2,0,88.6,168.8,64.1,...,5,3.47,2.68,9.0,111,5000,21,27,13495.0,0
1,3,1,0,2,0,2,0,88.6,168.8,64.1,...,5,3.47,2.68,9.0,111,5000,21,27,16500.0,0
2,1,1,0,2,2,2,0,94.5,171.2,65.5,...,5,2.68,3.47,9.0,154,5000,19,26,16500.0,0
3,2,1,0,4,3,1,0,99.8,176.6,66.2,...,5,3.19,3.40,10.0,102,5500,24,30,13950.0,1
4,2,1,0,4,3,0,0,99.4,176.6,66.4,...,5,3.19,3.40,8.0,115,5500,18,22,17450.0,1


random_null :  a function which helps to randomly change values to null according to the percentage we mention 

In [ ]:
def random_null(df, column_list, percentage):
    df_copy  = df
    percentage = percentage / 100
    total_rows = df_copy.shape[0]
    
    for column_name in column_list:
        
        num_nulls = int(total_rows * percentage)
        null_indices = np.random.choice(df_copy.index, num_nulls, replace= False)
        df_copy.loc[null_indices, column_name] = np.nan
        
    return df_copy

In [11]:
data_columns = list(data.columns)

In [12]:
data_columns

['symboling',
 'fueltype',
 'aspiration',
 'doornumber',
 'carbody',
 'drivewheel',
 'enginelocation',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginetype',
 'cylindernumber',
 'enginesize',
 'fuelsystem',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg',
 'price',
 'company_name']